In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 8: Time series
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

## Submission instructions
<hr>
rubric={points:4}

You will receive marks for correctly submitting this assignment. 

Follow the homework instructions below and at the end of this file. 
There are also detailed [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023s/blob/main/docs/homework_instructions.md) on github.

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture, we had different measurements for each Location. How about this avocado sales dataset? For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

In [9]:
df["region"].unique().shape

(54,)

In [10]:
df["type"].unique()

array(['conventional', 'organic'], dtype=object)

In [11]:
df["year"].unique()

array([2015, 2016, 2017, 2018], dtype=int64)

In [12]:
df.head(10)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.0,conventional,2015,Albany
7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,conventional,2015,Albany
8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,0.0,conventional,2015,Albany
9,2015-10-25,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,0.0,conventional,2015,Albany


In [13]:
df.iloc[9127:9137]

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
1,2015-12-20,1.89,1163.03,30.24,172.14,0.0,960.65,960.65,0.00,0.0,organic,2015,Albany
2,2015-12-13,1.85,995.96,10.44,178.70,0.0,806.82,806.82,0.00,0.0,organic,2015,Albany
3,2015-12-06,1.84,1158.42,90.29,104.18,0.0,963.95,948.52,15.43,0.0,organic,2015,Albany
4,2015-11-29,1.94,831.69,0.00,94.73,0.0,736.96,736.96,0.00,0.0,organic,2015,Albany
5,2015-11-22,1.94,858.83,13.84,84.18,0.0,760.81,755.69,5.12,0.0,organic,2015,Albany
6,2015-11-15,1.89,1208.54,20.71,238.16,0.0,949.67,949.67,0.00,0.0,organic,2015,Albany
7,2015-11-08,1.88,1332.27,20.08,351.40,0.0,960.79,960.79,0.00,0.0,organic,2015,Albany
8,2015-11-01,1.88,1021.68,11.47,137.58,0.0,872.63,872.63,0.00,0.0,organic,2015,Albany
9,2015-10-25,1.83,1161.90,49.27,148.96,0.0,963.67,963.67,0.00,0.0,organic,2015,Albany
10,2015-10-18,1.97,969.29,10.31,158.07,0.0,800.91,800.91,0.00,0.0,organic,2015,Albany


We would have separate measurements for the regions category, and optionally the type category if we want to separate the relevant information of organic avocados because their numbers differ drastically.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

Yes, the measurements are equally spaced.  We can see that the dates are exactly one week apart each from 2015 to 2018.

In [14]:
df["Date"].unique()

array(['2015-12-27T00:00:00.000000000', '2015-12-20T00:00:00.000000000',
       '2015-12-13T00:00:00.000000000', '2015-12-06T00:00:00.000000000',
       '2015-11-29T00:00:00.000000000', '2015-11-22T00:00:00.000000000',
       '2015-11-15T00:00:00.000000000', '2015-11-08T00:00:00.000000000',
       '2015-11-01T00:00:00.000000000', '2015-10-25T00:00:00.000000000',
       '2015-10-18T00:00:00.000000000', '2015-10-11T00:00:00.000000000',
       '2015-10-04T00:00:00.000000000', '2015-09-27T00:00:00.000000000',
       '2015-09-20T00:00:00.000000000', '2015-09-13T00:00:00.000000000',
       '2015-09-06T00:00:00.000000000', '2015-08-30T00:00:00.000000000',
       '2015-08-23T00:00:00.000000000', '2015-08-16T00:00:00.000000000',
       '2015-08-09T00:00:00.000000000', '2015-08-02T00:00:00.000000000',
       '2015-07-26T00:00:00.000000000', '2015-07-19T00:00:00.000000000',
       '2015-07-12T00:00:00.000000000', '2015-07-05T00:00:00.000000000',
       '2015-06-28T00:00:00.000000000', '2015-06-21

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [15]:
df["region"].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

There are many overlapping regions.  For example, SanFrancisco, California, and TotalUS are all present, and SanFrancisco is in California and California is in the US.

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023s/blob/main/lectures/19_time-series.ipynb), with some improvements.

In [16]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [17]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [18]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [19]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

### 1.4 `AveragePrice` baseline 
rubric={autograde:4}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [20]:


train_r2 = r2_score(df_train["AveragePriceNextWeek"], df_train["AveragePrice"])
train_r2

0.8285800937261841

In [21]:
test_r2 = r2_score(df_test["AveragePriceNextWeek"], df_test["AveragePrice"])
test_r2

0.7631780188583048

In [22]:
grader.check("q1.4")

q1.4 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 10

In [23]:
y_train = df_train["AveragePriceNextWeek"]
y_test = df_test["AveragePriceNextWeek"]

In [24]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [25]:
# adaptation of score_lr_print_coeff() from L19
def print_lr_score(preprocessor, df_train=df_train, df_test=df_test):
    lr_pipe = make_pipeline(preprocessor, Ridge(max_iter=1000))
    lr_pipe.fit(df_train, y_train)
    print("Train score: {:.4f}".format(lr_pipe.score(df_train, y_train)))
    print("Test score: {:.4f}".format(lr_pipe.score(df_test, y_test)))

In [26]:
df_train.columns

Index(['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770',
       'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year',
       'region', 'AveragePriceNextWeek'],
      dtype='object')

### 1. Drop Date

In [27]:
num = ['AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'year']
binary = ['type']
ohe = ['region']

In [28]:
pp_drop = make_column_transformer(
    (StandardScaler(), num),
    (OneHotEncoder(drop="if_binary", sparse="false"), binary),
    (OneHotEncoder(sparse="false", handle_unknown="ignore"), ohe),
    ("drop", ['Date'])
)

In [29]:
print_lr_score(pp_drop)

Train score: 0.8451
Test score: 0.7882


### 2. Encode Date as number of days since first day

In [30]:
first_day = df_train["Date"].min()
df_train_since = df_train.assign(
    Days_since=df_train["Date"].apply(lambda x: (x - first_day).days)
)
df_test_since = df_test.assign(
    Days_since=df_test["Date"].apply(lambda x: (x - first_day).days)
)

In [31]:
pp_since = make_column_transformer(
    (StandardScaler(), num + ['Days_since']),
    (OneHotEncoder(drop="if_binary", sparse="false"), binary),
    (OneHotEncoder(sparse="false", handle_unknown="ignore"), ohe),
    ("drop", ['Date'])
)

In [32]:
print_lr_score(pp_since, df_train_since, df_test_since)

Train score: 0.8457
Test score: 0.7841


### 3. OHE on month

In [33]:
df_train_month = df_train.assign(
    Month=df_train["Date"].apply(lambda x: x.month_name())
)
df_test_month = df_test.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))

In [34]:
pp_month = make_column_transformer(
    (StandardScaler(), num),
    (OneHotEncoder(drop="if_binary", sparse="false"), binary),
    (OneHotEncoder(sparse="false", handle_unknown="ignore"), ohe + ['Month']),
    ("drop", ['Date'])
)

In [35]:
print_lr_score(pp_month, df_train_month, df_test_month)

Train score: 0.8494
Test score: 0.8003


### 4. Both Days_since and OHE on month

In [36]:
df_train_since_month = df_train_since.assign(
    Month=df_train["Date"].apply(lambda x: x.month_name())
)
df_test_since_month = df_test_since.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))

In [37]:
pp_since_month = make_column_transformer(
    (StandardScaler(), num + ['Days_since']),
    (OneHotEncoder(drop="if_binary", sparse="false"), binary),
    (OneHotEncoder(sparse="false", handle_unknown="ignore"), ohe + ['Month']),
    ("drop", ['Date'])
)

In [38]:
print_lr_score(pp_since_month, df_train_since_month, df_test_since_month)

Train score: 0.8494
Test score: 0.8004


### Decisions
- Performed standard scaling on all numerical features to equalize feature importance and normalize scales, since many of the values vary heavily in magnitude
- OHE on region since it was a categorical feature that wasn't continuous, and binary OHE on type since there were two options
1. Ignored date entirely to get a baseline score
2. Encoded date as number of days since the first day to see if it would make a difference.  Ended up slightly decreasing test score, indicating that it was not very useful and lacks an obvious trend over the years.
3. OHE on month to see if seasonality would be a factor.  Increased the test score by a significant amount, which tells us that the time of year plays a part in the price changes.
4. Tried both #2 and #3 combined to see if days since, in combination with months, might make a difference.  The idea is that while month is a significant factor, combining it with a time feature might give a better estimate.  A tiny increase of 0.0001 in the test score from #3, combined with the fact that #2 was ineffective, suggests that #2 is probably not necessary and the test score increasing from #3 may have been a fluke, but the combination could possibly be significant.

## Exercise 2: very short answer questions

Each question is worth 2 points.

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:4}

The following questions pertain to Lecture 19 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 4

1. An example could be a timestamp of every single avocado sale, i.e. record the exact time at which each avocado is sold.  While there will be patterns, this event would occur very sporadically and not at all at a consistent interval.
2. Lagging would struggle with unequal spacing since we create lag-based features by shifting in consistent time lag intervals.  With unequal spacing there would be issues in creating lags and also making predictions due to varying time intervals between observations.  Encoding the date as features would not struggle with unequal spacing because we can choose how to extract features, whether it be converting it to numbers like day counts or categorically encoding months and seasons.

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis
rubric={points:6}

The following questions pertain to [Lecture 20](https://github.com/UBC-CS/cpsc330-2023s/blob/main/lectures/20_survival-analysis.ipynb) on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

1. This fails to take into account all temporal factors such as average subscription durations, survival probability over time, and when they joined.  It also doesn't consider or provide insight into reasons why or when they may leave.
2. We don't have enough information, such as average tenure, survival probabilities, and external factors that influence customer behaviour.  For example, it could be the case that customers are highly likely to leave any time within the first month, but those who don't tend to stick around for years, OR it could be the case that customers tend to stay for roughly a year, then choose to leave due to any reasons we don't have information on, for example a steep price hike due to the end of a free trial or promotion.
3. It likely means that survival probability around that period is stable, i.e. customers are not more or less likely to leave within the interval.

<!-- END QUESTION -->

<br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)